# WESAD
###  A Multimodal Dataset for Wearable Stress and Affect Detection

##### Matthew Johnson, 2019

===============================================================================================================


### Dataset Information [1]:
Data Set Information:

WESAD is a publicly available dataset for wearable stress and affect detection. This multimodal dataset features physiological and motion data, recorded from both a wrist- and a chest-worn device, of 15 subjects during a lab study. The following sensor modalities are included: blood volume pulse, electrocardiogram, electrodermal activity, electromyogram, respiration, body temperature, and three-axis acceleration. Moreover, the dataset bridges the gap between previous lab studies on stress and emotions, by containing three different affective states (neutral, stress, amusement). In addition, self-reports of the subjects, which were obtained using several established questionnaires, are contained in the dataset. Details can be found in the dataset's readme-file, as well as in [1].


Attribute Information:

Raw sensor data was recorded with two devices: a chest-worn device (RespiBAN) and a wrist-worn device (Empatica E4). 
The RespiBAN device provides the following sensor data: electrocardiogram (ECG), electrodermal activity (EDA), electromyogram (EMG), respiration, body temperature, and three-axis acceleration. All signals are sampled at 700 Hz. 
The Empatica E4 device provides the following sensor data: blood volume pulse (BVP, 64 Hz), electrodermal activity (EDA, 4 Hz), body temperature (4 Hz), and three-axis acceleration (32 Hz). 

The dataset's readme-file contains all further details with respect to the dataset structure, data format (RespiBAN device, Empatica E4 device, synchronised data), study protocol, and the self-report questionnaires.


- https://archive.ics.uci.edu/ml/datasets/WESAD+%28Wearable+Stress+and+Affect+Detection%29



### Classes

**Baseline condition**: 20 minute period of standing/sitting reading magazines.<br>
**Amusement condition**: During the amusement condition, the
subjects watched a set of eleven funny video clips.<br>
**Stress condition**: Trier Social Stress Test (TSST), consisting of public speaking and mental arithmetic.




------------
   
#### References

[1] Schmidt, Philip & Reiss, Attila & Duerichen, Robert & Marberger, Claus & Van Laerhoven, Kristof. (2018). Introducing WESAD, a Multimodal Dataset for Wearable Stress and Affect Detection. 400-408. 10.1145/3242969.3242985.  https://dl.acm.org/citation.cfm?doid=3242969.3242985

[2] A Greco, G Valenza, A Lanata, EP Scilingo, and L Citi
"cvxEDA: a Convex Optimization Approach to Electrodermal Activity Processing"
IEEE Transactions on Biomedical Engineering, 2015
DOI: 10.1109/TBME.2015.2474131
https://github.com/lciti/cvxEDA

[3] J. Choi, B. Ahmed, and R. Gutierrez-Osuna. 2012. Development and evaluation
of an ambulatory stress monitor based on wearable sensors. IEEE Transactions
on Information Technology in Biomedicine 16, 2 (2012).  
    http://research.cs.tamu.edu/prism/publications/choi2011ambulatoryStressMonitor.pdf
    
[6] J. Healey and **R. Picard.** 2005. Detecting stress during real-world driving tasks
using physiological sensors. IEEE Transactions on Intelligent Transportation
Systems 6, 2 (2005), 156–166.  


#### Useful Resources:
- https://github.com/jaganjag/stress_affect_detection
- https://github.com/arsen-movsesyan/springboard_WESAD
- https://www.birmingham.ac.uk/Documents/college-les/psych/saal/guide-electrodermal-activity.pdf
- http://research.cs.tamu.edu/prism/publications/choi2011ambulatoryStressMonitor.pdf

 TODO: 
        - add early stopping?
        - maybe change to binary classification (stress, not-stress)?

## Dataset and Dataloading

In [44]:
!pip uninstall torch torchvision torchaudio -y
!pip cache purge
!pip install torch torchvision torchaudio

Found existing installation: torch 2.0.1
Uninstalling torch-2.0.1:
  Successfully uninstalled torch-2.0.1
Found existing installation: torchvision 0.15.2
Uninstalling torchvision-0.15.2:
  Successfully uninstalled torchvision-0.15.2
Found existing installation: torchaudio 2.0.2
Uninstalling torchaudio-2.0.2:
  Successfully uninstalled torchaudio-2.0.2
Files removed: 48 (63.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.1 MB/s eta 0:00:00a 0:00:01


In [27]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset
from sklearn.metrics import confusion_matrix




In [2]:
class WESADDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe.drop('subject', axis=1)
        self.labels = self.dataframe['label'].values
        self.dataframe.drop('label', axis=1, inplace=True)
        
    def __getitem__(self, idx):
        x = self.dataframe.iloc[idx].values
        y = self.labels[idx]
        return torch.Tensor(x), y

    def __len__(self):
        return len(self.dataframe)

### Cross Validation Loader

In [6]:
feats =   ['BVP_mean', 'BVP_std', 'BVP_min', 'BVP_max',
           'EDA_phasic_mean', 'EDA_phasic_std', 'EDA_phasic_min', 'EDA_phasic_max', 'EDA_smna_mean',
           'EDA_smna_std', 'EDA_smna_min', 'EDA_smna_max', 'EDA_tonic_mean',
           'EDA_tonic_std', 'EDA_tonic_min', 'EDA_tonic_max', 'Resp_mean',
           'Resp_std', 'Resp_min', 'Resp_max', 'TEMP_mean', 'TEMP_std', 'TEMP_min',
           'TEMP_max', 'TEMP_slope', 'BVP_peak_freq', 'age', 'height',
           'weight','subject', 'label']
layer_1_dim = len(feats) -2
print(layer_1_dim)

29


In [7]:
def get_data_loaders(df, subject_id, train_batch_size=25, test_batch_size=5):
    #df = pd.read_csv('data/m14_merged.csv', index_col=0)[feats]

    train_df = df[ df['subject'] != subject_id].reset_index(drop=True)
    test_df = df[ df['subject'] == subject_id].reset_index(drop=True)
    
    train_dset = WESADDataset(train_df)
    test_dset = WESADDataset(test_df)

    train_loader = torch.utils.data.DataLoader(train_dset, batch_size=train_batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dset, batch_size=test_batch_size)
    
    return train_loader, test_loader

## Network Architecture

In [8]:
class StressNet(nn.Module):
    def __init__(self):
        super(StressNet, self).__init__()
        self.fc = nn.Sequential(
                        nn.Linear(29, 128),
                        #nn.Dropout(0.5),
                        nn.ReLU(),
                        nn.Linear(128, 256),
                        #nn.Dropout(0.5),
                        nn.ReLU(),
                        nn.Linear(256, 2),
                        #nn.Dropout(0.5),
                        nn.LogSoftmax(dim=1))
        
    def forward(self, x):
        return self.fc(x)    

## Model Training

In [9]:
def train(model, optimizer, train_loader, validation_loader):
    history = {'train_loss': {}, 'train_acc': {}, 'valid_loss': {}, 'valid_acc': {}}
    #
    for epoch in range(num_epochs):

        # Train:   
        total = 0
        correct = 0
        trainlosses = []

        for batch_index, (images, labels) in enumerate(train_loader):

            # Send to GPU (device)
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images.float())

            # Loss
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            trainlosses.append(loss.item())

            # Compute accuracy
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item() #.mean()
            total += len(labels)

        history['train_loss'][epoch] = np.mean(trainlosses) 
        history['train_acc'][epoch] = correct/total 

        if epoch % 10 == 0:
            with torch.no_grad():

                losses = []
                total = 0
                correct = 0

                for images, labels in validation_loader:
                    # 
                    images, labels = images.to(device), labels.to(device)

                    # Forward pass
                    outputs = model(images.float())
                    loss = criterion(outputs, labels)

                    # Compute accuracy
                    _, argmax = torch.max(outputs, 1)
                    correct += (labels == argmax).sum().item() #.mean()
                    total += len(labels)

                    losses.append(loss.item())
                    
                history['valid_acc'][epoch] = np.round(correct/total, 3)
                history['valid_loss'][epoch] = np.mean(losses)

                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {np.mean(losses):.4}, Acc: {correct/total:.2}')
                
    return history

In [10]:
def test(model, validation_loader):
    print('Evaluating model...')
    # Test
    model.eval()

    total = 0
    correct = 0
    testlosses = []
    correct_labels = []
    predictions = []

    with torch.no_grad():

        for batch_index, (images, labels) in enumerate(validation_loader):
            # Send to GPU (device)
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images.float())

            # Loss
            loss = criterion(outputs, labels)

            testlosses.append(loss.item())

            # Compute accuracy
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item() #.mean()
            total += len(labels)

            correct_labels.extend(labels)
            predictions.extend(argmax)


    test_loss = np.mean(testlosses)
    accuracy = np.round(correct/total, 2)
    print(f'Loss: {test_loss:.4}, Acc: {accuracy:.2}')
    
    y_true = [label.item() for label in correct_labels]
    y_pred = [label.item() for label in predictions]

    # TODO: return y true and y pred, make cm after ( use ytrue/ypred for classification report)
    return y_true, y_pred, test_loss, accuracy
    #return cm, test_loss, accuracy

## Start Here

In [12]:
df = pd.read_csv('data/m14_merged.csv', index_col=0)
subject_id_list = df['subject'].unique()
df.head()

,net_acc_mean,net_acc_std,net_acc_min,net_acc_max,EDA_phasic_mean,EDA_phasic_std,EDA_phasic_min,EDA_phasic_max,EDA_smna_mean,EDA_smna_std,...,age,height,weight,gender_ female,gender_ male,coffee_today_YES,sport_today_YES,smoker_NO,smoker_YES,feel_ill_today_YES
0,0.609529,0.141481,-358.13,554.77,0.609529,1.089131,-358.13,554.77,0.609529,1.952141,...,27,175,80,False,True,False,False,True,False,False
1,0.538008,0.091882,-392.28,438.16,0.538008,1.223623,-392.28,438.16,0.538008,2.854162,...,27,175,80,False,True,False,False,True,False,False
2,0.574784,0.102315,-240.61,209.89,0.574784,0.129121,-240.61,209.89,0.574784,0.245284,...,27,175,80,False,True,False,False,True,False,False
3,0.590880,0.046391,-289.26,145.36,0.590880,0.126564,-289.26,145.36,0.590880,0.201035,...,27,175,80,False,True,False,False,True,False,False
4,0.567452,0.034540,-197.37,194.12,0.567452,0.039667,-197.37,194.12,0.567452,0.115003,...,27,175,80,False,True,False,False,True,False,False


In [13]:
def change_label(label):
    if label == 0 or label == 1:
        return 0
    else:
        return 1
    
df['label'] = df['label'].apply(change_label)

In [14]:
df = df[feats]

In [15]:
# Batch sizes
train_batch_size = 25
test_batch_size = 5

# Learning Rate
learning_rate = 5e-3

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Number of Epochs
num_epochs = 100

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

# models = [] # save models at all/ directly?
y_preds = []
y_truths = []
histories = []
confusion_matrices = []
test_losses = []
test_accs = []

for _ in subject_id_list:
    print('\nSubject: ', _)
    model = StressNet().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    train_loader, test_loader = get_data_loaders(df, _)
    
    history = train(model, optimizer, train_loader, test_loader)
    histories.append(history)
    
    y_true, y_pred, test_loss, test_acc = test(model, test_loader)
    
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    #confusion_matrices.append(cm)
    y_preds.append(y_pred)
    y_truths.append(y_true)


Subject:  2
Epoch [1/100], Loss: 0.0, Acc: 1.0
Epoch [11/100], Loss: 0.0, Acc: 1.0
Epoch [21/100], Loss: 0.0, Acc: 1.0
Epoch [31/100], Loss: 0.0, Acc: 1.0
Epoch [41/100], Loss: 0.0, Acc: 1.0
Epoch [51/100], Loss: 0.0, Acc: 1.0
Epoch [61/100], Loss: 0.0, Acc: 1.0
Epoch [71/100], Loss: 0.0, Acc: 1.0
Epoch [81/100], Loss: 0.0, Acc: 1.0
Epoch [91/100], Loss: 0.0, Acc: 1.0
Evaluating model...
Loss: 0.0, Acc: 1.0

Subject:  3
Epoch [1/100], Loss: 0.0, Acc: 1.0
Epoch [11/100], Loss: 0.0, Acc: 1.0
Epoch [21/100], Loss: 0.0, Acc: 1.0
Epoch [31/100], Loss: 0.0, Acc: 1.0
Epoch [41/100], Loss: 0.0, Acc: 1.0
Epoch [51/100], Loss: 0.0, Acc: 1.0
Epoch [61/100], Loss: 0.0, Acc: 1.0
Epoch [71/100], Loss: 0.0, Acc: 1.0
Epoch [81/100], Loss: 0.0, Acc: 1.0
Epoch [91/100], Loss: 0.0, Acc: 1.0
Evaluating model...
Loss: 0.0, Acc: 1.0

Subject:  4
Epoch [1/100], Loss: 0.0, Acc: 1.0
Epoch [11/100], Loss: 0.0, Acc: 1.0
Epoch [21/100], Loss: 0.0, Acc: 1.0
Epoch [31/100], Loss: 0.0, Acc: 1.0
Epoch [41/100], Loss

In [16]:
print('Mean Accuracy:', np.mean(test_accs))
print('Accuracy std:', np.std(test_accs))

Mean Accuracy: 1.0
Accuracy std: 0.0


In [17]:
np.mean(test_losses)

0.0

In [18]:
df['label'].value_counts()

label
1    1178
Name: count, dtype: int64

In [19]:
plt.figure(figsize=(14, 6))
plt.title('Testing Accuracies in Leave One Out Cross Validation by Subject Left Out as Testing Data')
sns.barplot(x=subject_id_list, y=test_accs);

In [20]:
plt.figure(figsize=(14, 3))
plt.title('Testing Losses in Leave One Out Cross Validation by Subject Left Out as Testing Data')
sns.barplot(x=subject_id_list, y=test_losses);

In [22]:
# infodf = pd.read_csv('data/WESAD/readmes.csv', index_col=0)
# infodf.sort_index()

## Training Visualization

In [23]:
len(histories)

15

## Model Evaluation

In [24]:
confusion_matrices = [confusion_matrix(y_true, y_pred) 
                     for y_true, y_pred in zip(y_truths, y_preds)]

plt.figure(figsize=(15,10))


for i in range(len(confusion_matrices)):
    plt.subplot(4,5 ,i+1)
    cm = confusion_matrices[i]
    
    
    sns.heatmap(cm, annot=True, fmt='d', cbar=False);
    plt.title(f'S{subject_id_list[i]}')
    plt.xlabel('Prediction');
    plt.ylabel('Ground Truth');
plt.tight_layout();

In [28]:
from sklearn.metrics import classification_report

target_names = ['Stress']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      Stress       1.00      1.00      1.00        76

    accuracy                           1.00        76
   macro avg       1.00      1.00      1.00        76
weighted avg       1.00      1.00      1.00        76



In [29]:
torch.save(model.state_dict(), 'm13_model.pt')

In [31]:
cr = classification_report(y_true, y_pred, 
                          target_names=target_names,
                          labels=[0, 1])  # Assuming 0=Not-Stress, 1=Stress

In [32]:
cr

'              precision    recall  f1-score   support\n\n  Not-Stress       0.00      0.00      0.00         0\n      Stress       1.00      1.00      1.00        76\n\n    accuracy                           1.00        76\n   macro avg       0.50      0.50      0.50        76\nweighted avg       1.00      1.00      1.00        76\n'

In [36]:
from sklearn.metrics import classification_report
classif_reports = []
for i, (y_true, y_pred) in enumerate(zip(y_truths, y_preds)):
    print('Subject', subject_id_list[i], ':')
    target_names = ['Not-Stress', 'Stress']
    cr = classification_report(y_true, y_pred, target_names=target_names)
    classif_reports.append(cr)
    print(cr)
    print()

Subject 2 :


ValueError: Number of classes, 1, does not match size of target_names, 2. Try specifying the labels parameter

In [37]:
from sklearn.metrics import classification_report
import numpy as np

classif_reports = []
for i, (y_true, y_pred) in enumerate(zip(y_truths, y_preds)):
    print('Subject', subject_id_list[i], ':')
    target_names = ['Not-Stress', 'Stress']
    
    # Get unique classes present in both true labels and predictions
    unique_classes = np.unique(np.concatenate([y_true, y_pred]))
    
    # Use labels parameter to explicitly specify which labels to include in report
    # This handles cases where not all classes are present
    try:
        cr = classification_report(y_true, y_pred, 
                                  target_names=target_names,
                                  labels=range(len(target_names)),  # Include all possible labels
                                  zero_division=0)  # Handle division by zero for missing classes
        classif_reports.append(cr)
        print(cr)
    except Exception as e:
        print(f"Error generating report: {e}")
        # Create a simple report for the special case
        if len(unique_classes) == 1:
            class_idx = unique_classes[0]
            class_name = target_names[class_idx]
            print(f"Only one class present: {class_name} (support: {len(y_true)})")
            classif_reports.append(f"Only class {class_name}, accuracy: 1.00, support: {len(y_true)}")
    print()

Subject 2 :
              precision    recall  f1-score   support

  Not-Stress       0.00      0.00      0.00         0
      Stress       1.00      1.00      1.00        76

    accuracy                           1.00        76
   macro avg       0.50      0.50      0.50        76
weighted avg       1.00      1.00      1.00        76


Subject 3 :
              precision    recall  f1-score   support

  Not-Stress       0.00      0.00      0.00         0
      Stress       1.00      1.00      1.00        77

    accuracy                           1.00        77
   macro avg       0.50      0.50      0.50        77
weighted avg       1.00      1.00      1.00        77


Subject 4 :
              precision    recall  f1-score   support

  Not-Stress       0.00      0.00      0.00         0
      Stress       1.00      1.00      1.00        76

    accuracy                           1.00        76
   macro avg       0.50      0.50      0.50        76
weighted avg       1.00      1.00   

In [34]:
f1_scores = [ float(cr.split('\n')[3].strip().split('      ')[3]) for cr in classif_reports]

In [38]:
print(np.mean(f1_scores))
print(np.std(f1_scores))

1.0
0.0
